In [8]:
import pandas as pd

df = pd.read_excel('../dados/localizacao_servicos_saude.xlsx')

df.head()

,ID,COORD_E,COORD_N,SETCENS,AREAP,CODDIST,DISTRITO,CODSUBPREF,SUBPREF,REGIAO5,...,TELEFONE,CEP,CNES,SA_DEPADM,DEPADM,SA_TIPO,TIPO,SA_CLASSE,CLASSE,LEITOS
0,1,319108.42,7380764.47,355030819000016,3550308005232,19,CAPAO REDONDO,17,CAMPO LIMPO,Sul,...,58729902,5866000,6194974,1,Municipal,49,AMA ESPECIALIDADES,1,AMBULATORIOS ESPECIALIZADOS,NaN
1,2,331055.43,7400384.23,355030821000001,3550308005062,21,CASA VERDE,4,CASA VERDE-CACHOEIRINHA,Norte,...,38589865,2538010,6148336,1,Municipal,49,AMA ESPECIALIDADES,1,AMBULATORIOS ESPECIALIZADOS,NaN
2,3,330196.17,7380955.02,355030822000066,3550308005164,22,CIDADE ADEMAR,16,CIDADE ADEMAR,Sul,...,55626944,4438280,6415415,1,Municipal,49,AMA ESPECIALIDADES,1,AMBULATORIOS ESPECIALIZADOS,NaN
3,4,326183.88,7373067.87,355030823000196,3550308005287,23,CIDADE DUTRA,19,CAPELA DO SOCORRO,Sul,...,59280272,4837150,6360890,1,Municipal,49,AMA ESPECIALIDADES,1,AMBULATORIOS ESPECIALIZADOS,NaN
4,5,351434.05,7396009.19,355030837000019,3550308005202,37,ITAQUERA,27,ITAQUERA,Leste,...,20712422,8210090,6394558,1,Municipal,49,AMA ESPECIALIDADES,1,AMBULATORIOS ESPECIALIZADOS,NaN


In [12]:
from pyproj import Proj

# Definindo o sistema de coordenadas UTM zona 23S
utm_proj = Proj(proj='utm', zone=23, south=True, ellps='WGS84')

# Função para converter UTM para Latitude e Longitude
def utm_para_latlon(leste, norte):
    lon, lat = utm_proj(leste, norte, inverse=True)
    return lat, lon

# Aplicar a função para cada linha do DataFrame e criar novas colunas 'Latitude' e 'Longitude'
df['Latitude'], df['Longitude'] = zip(*df.apply(lambda row: utm_para_latlon(row['COORD_E'], row['COORD_N']), axis=1))

# Exibindo o DataFrame atualizado
df.head()


,ID,COORD_E,COORD_N,SETCENS,AREAP,CODDIST,DISTRITO,CODSUBPREF,SUBPREF,REGIAO5,...,CNES,SA_DEPADM,DEPADM,SA_TIPO,TIPO,SA_CLASSE,CLASSE,LEITOS,Latitude,Longitude
0,1,319108.42,7380764.47,355030819000016,3550308005232,19,CAPAO REDONDO,17,CAMPO LIMPO,Sul,...,6194974,1,Municipal,49,AMA ESPECIALIDADES,1,AMBULATORIOS ESPECIALIZADOS,NaN,-23.673787,-46.773848
1,2,331055.43,7400384.23,355030821000001,3550308005062,21,CASA VERDE,4,CASA VERDE-CACHOEIRINHA,Norte,...,6148336,1,Municipal,49,AMA ESPECIALIDADES,1,AMBULATORIOS ESPECIALIZADOS,NaN,-23.497937,-46.654510
2,3,330196.17,7380955.02,355030822000066,3550308005164,22,CIDADE ADEMAR,16,CIDADE ADEMAR,Sul,...,6415415,1,Municipal,49,AMA ESPECIALIDADES,1,AMBULATORIOS ESPECIALIZADOS,NaN,-23.673273,-46.665135
3,4,326183.88,7373067.87,355030823000196,3550308005287,23,CIDADE DUTRA,19,CAPELA DO SOCORRO,Sul,...,6360890,1,Municipal,49,AMA ESPECIALIDADES,1,AMBULATORIOS ESPECIALIZADOS,NaN,-23.744055,-46.705393
4,5,351434.05,7396009.19,355030837000019,3550308005202,37,ITAQUERA,27,ITAQUERA,Leste,...,6394558,1,Municipal,49,AMA ESPECIALIDADES,1,AMBULATORIOS ESPECIALIZADOS,NaN,-23.539435,-46.455426


In [14]:
import folium
from folium.plugins import MarkerCluster

# Criando um mapa inicial com as coordenadas de SP
map = folium.Map(location=[-23.5505, -46.6333], zoom_start=10)

# Adicionando um marcador ao mapa
marker_cluster = MarkerCluster().add_to(map)

# Iterando pelas Lat/Lon para adicionar ao mapa
for index, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']], popup=row['ESTAB_SA']).add_to(map)
map